<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myProspects_Upside_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance==0.2.59

In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/quantvesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Mounted at /content/drive


In [3]:
# Import the function from the script
import myProspects_Upside_Analysis_v2

all = 1

# Call the function to get the DataFrame
df_prospects_features = myProspects_Upside_Analysis_v2.get_prospects_features_v2(all)
cols = ['Symbol', 'Close', 'FTT',  'Dev%_200', 'Dev%_PE', 'Conviction', 'RSP', 'Today%', 'FTT%', 'ATH%','Gained%', 'Criteria', 'Strategy', 'Sales_Grwth%', 'Profit_Grwth%', 'Category', 'InFolio', 'MCap', 'MBQ', 'ROE%', 'CFO_2_EBITDA%', 'ROE%/PE']
df_prospects_features = df_prospects_features[cols].drop_duplicates()

Run date time (IST): 2025-05-14 14:08:27
-------------------


ERROR:yfinance:$ISEC.NS: possibly delisted; no price data found  (period=1mo)


qualified stocks: 260
with latest results: 60
still star stocks: 42


In [4]:
cols = ['Symbol', 'FTT', 'Dev%_200', 'Dev%_PE', 'Conviction', 'RSP', 'FTT%', 'ATH%','Gained%', 'CumlRnk', 'Criteria', 'Strategy', 'Category', 'InFolio']

tmp_df = df_prospects_features
list_x40 = list(tmp_df[tmp_df['MBQ'].str.contains('X40', na=False)]['Symbol'].values)
list_x5k = list(tmp_df[tmp_df['MBQ'].str.contains('X5K', na=False)]['Symbol'].values)
list_x40n = list(tmp_df[tmp_df['MBQ'].str.contains('X40N', na=False)]['Symbol'].values)
list_hc = list(tmp_df[tmp_df['Conviction'].isin(['X-LC', 'H-LC', 'X-MC', 'X-SC'])]['Symbol'].values)
#len(set(list_x40 + list_x5k) - set( list_x40n))
stock_list = list(set(list_x40 + list_x5k + list_hc))

In [6]:
# get the cumulative rank
df_tmp = df_prospects_features # [df_prospects_features['Symbol'].isin(stock_list)]

df_tmp['Rnk_Dev%_200'] = df_tmp['Dev%_200'].rank(method='min', ascending=True)
df_tmp['Rnk_Dev%_PE'] = df_tmp['Dev%_PE'].rank(method='min', ascending=True)
df_tmp['Rnk_Gained%'] = df_tmp['Gained%'].rank(method='min', ascending=True)
df_tmp['Rnk_ATH%'] = df_tmp['ATH%'].rank(method='max', ascending=False)
df_tmp['Rnk_Sales_Grwth%'] = df_tmp['Sales_Grwth%'].rank(method='max', ascending=False)
df_tmp['Rnk_Profit_Grwth%'] = df_tmp['Profit_Grwth%'].rank(method='max', ascending=False)
df_tmp['Rnk_MCap'] = df_tmp['MCap'].rank(method='max', ascending=False)
df_tmp['Rnk_ROE%'] = df_tmp['ROE%'].rank(method='max', ascending=False)
df_tmp['Rnk_RSP'] = df_tmp['RSP'].rank(method='max', ascending=False)
df_tmp['Rnk_CFO_2_EBITDA%'] = df_tmp['CFO_2_EBITDA%'].rank(method='max', ascending=False)
df_tmp['Rnk_ROE%/PE'] = df_tmp['ROE%/PE'].rank(method='max', ascending=False)
df_tmp['Ovrl_Rank'] = df_tmp['Rnk_Dev%_200'] + df_tmp['Rnk_Dev%_PE'] + df_tmp['Rnk_Gained%'] + df_tmp['Rnk_ATH%'] + df_tmp['Rnk_Sales_Grwth%'] + df_tmp['Rnk_Profit_Grwth%'] + df_tmp['Rnk_MCap'] + df_tmp['Rnk_CFO_2_EBITDA%'] + df_tmp['Rnk_ROE%'] + df_tmp['Rnk_RSP'] + df_tmp['Rnk_ROE%/PE']
df_tmp['CumlRnk'] = df_tmp['Ovrl_Rank'].rank(method='min', ascending=True)
# Drop the helper columns
df_tmp.drop(columns=['Rnk_Dev%_200','Rnk_Dev%_PE','Rnk_Gained%','Rnk_ATH%','Rnk_Sales_Grwth%','Rnk_Profit_Grwth%','Rnk_MCap','Rnk_ROE%','Rnk_RSP','Rnk_CFO_2_EBITDA%'], inplace=True)
df_prospects_features = df_tmp[cols]
data_table.DataTable(df_prospects_features.sort_values(by = 'CumlRnk', ascending=True), include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSP,FTT%,ATH%,Gained%,CumlRnk,Criteria,Strategy,Category,InFolio
320,TATAMOTORS,1065.00,-14.69,-53.87,M-LC,83.01,52.61,66.49,20.37,1.0,XY24,NTT,AUTO,1.0
226,NATIONALUM,247.44,-14.04,-44.96,L-SC,87.64,49.93,49.93,15.24,2.0,NaN,NaN,METALS,NaN
159,INDIAMART,4911.36,-6.69,-53.52,H-SC,63.32,115.19,115.19,18.52,3.0,AR,ATH,MISC,1.0
4,ABB,8953.12,-16.18,-38.61,H-LC,67.57,59.42,59.42,14.48,4.0,AR,ATH,ELECTRICAL,1.0
323,TCS,4998.00,-11.84,-15.19,X-LC,76.06,40.85,27.72,9.81,5.0,X40,BTT,IT,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,SPORTKING,157.26,8.16,NaN,NaN,100.00,37.17,37.17,59.95,NaN,AR,ATH,TEXTILES,NaN
316,TASTYBITE,20387.66,-7.42,NaN,NaN,96.53,107.61,107.61,26.13,NaN,XR,ATH,FMCG,NaN
343,V2RETAIL,2031.65,15.62,NaN,NaN,9.27,20.93,20.93,268.34,NaN,AR,ATH,TEXTILES,NaN
354,VEEDOL,3065.39,-13.79,NaN,NaN,66.80,93.88,93.88,16.54,NaN,XR,ATH,CHEMICALS,NaN


In [7]:
# module to print date timestamp
def read_csv(csv_file_name):
  csv_file_path = os.path.join(project_path, 'data', csv_file_name)
  df = pd.read_csv(csv_file_path)
  return df

mypps_df = read_csv('myProspectsScrips.csv')

# Suppose this is your updated DataFrame with new cuml_rnk values
cols1 = ['Symbol', 'CumlRnk']
df_tmp = df_prospects_features[cols1]

# Merge the two DataFrames on 'symbol', keeping the CSV structure
mypps_df = mypps_df.merge(df_tmp[['Symbol', 'CumlRnk']], on='Symbol', how='left', suffixes=('', '_new'))

# Replace 'cuml_rnk' with the new value only if it exists in df
mypps_df['CumlRnk'] = mypps_df['CumlRnk_new'].combine_first(mypps_df['CumlRnk'])

# Drop the helper column
mypps_df.drop(columns=['CumlRnk_new'], inplace=True)

# Save back to CSV
#mypps_df.to_csv(project_path+'/data/myProspectsScrips.csv', index=False)

In [8]:
# High quality (X40 + X40N + X5K + X-LC + H-LC + X-MC + X-SC) prospects
df_tmp = df_prospects_features[df_prospects_features['Symbol'].isin(stock_list)]
data_table.DataTable(df_tmp.sort_values(by = 'CumlRnk', ascending=True), include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSP,FTT%,ATH%,Gained%,CumlRnk,Criteria,Strategy,Category,InFolio
320,TATAMOTORS,1065.00,-14.69,-53.87,M-LC,83.01,52.61,66.49,20.37,1.0,XY24,NTT,AUTO,1.0
4,ABB,8953.12,-16.18,-38.61,H-LC,67.57,59.42,59.42,14.48,4.0,AR,ATH,ELECTRICAL,1.0
323,TCS,4998.00,-11.84,-15.19,X-LC,76.06,40.85,27.72,9.81,5.0,X40,BTT,IT,1.0
200,LICI,983.00,-9.12,-16.72,H-LC,55.21,19.26,44.14,13.03,6.0,XY25,NTT,INSURANCE,1.0
250,PNB,232.89,-5.29,-60.16,L-LC,26.06,138.74,138.74,12.26,9.0,XR,ATH,BANKS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,GRASIM,2832.86,5.13,144.35,H-LC,23.94,3.76,3.76,21.46,230.0,AR,ATH,TEXTILES,NaN
302,SRF,3055.70,11.87,102.84,X-MC,8.11,6.20,6.20,34.39,231.0,XR,ATH,CHEMICALS,NaN
296,SHREECEM,31254.03,14.58,84.63,X-LC,17.37,1.79,1.79,28.51,234.0,NaN,ATH,CEMENT,NaN
161,INDIGO,5558.50,19.29,48.97,M-LC,48.65,1.58,1.58,42.17,NaN,AR,ATH,AIRLINES,NaN


In [9]:
# x24/x25
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XY24', 'X25'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSP,FTT%,ATH%,Gained%,CumlRnk,Criteria,Strategy,Category,InFolio
255,QUESS,986.0,-43.44,NaN,NaN,99.23,168.63,226.18,23.09,NaN,XY24,NTT,MISC,1.0
48,BANDHANBNK,400.0,-5.04,-37.04,H-SC,71.81,146.34,339.10,24.61,30.0,XY24,NTT,BANKS,1.0
270,REPCOHOME,880.0,-7.99,-56.78,H-SC,80.31,118.36,109.24,29.48,86.0,XY24,NTT,FINANCE,1.0
103,EASEMYTRIP,26.4,-23.75,23.51,M-SC,2.70,116.39,179.59,10.51,28.0,XY24,NTT,TRAVEL,1.0
128,GREENPANEL,537.0,-24.64,174.55,M-SC,57.92,114.00,138.98,15.19,105.0,XY24,NTT,MISC,1.0
7,ACC,3906.0,-13.51,-36.77,M-MC,1.54,109.85,48.94,2.89,79.0,XY24,BTT,CEMENT,1.0
326,TITAGARH,1548.0,-26.45,-20.59,H-SC,25.48,94.16,133.14,18.09,98.0,XY24,NTT,ENGINEERING,1.0
290,SFL,1287.0,-21.84,279.94,H-SC,39.38,93.50,196.53,3.26,56.0,XY24,NTT,MISC,1.0
303,STARHEALTH,761.0,-16.88,-10.95,M-SC,52.51,89.94,126.34,16.77,65.0,XY24,NTT,INSURANCE,1.0
276,SAIL,228.0,2.34,84.51,M-MC,66.41,87.21,64.72,21.77,221.0,XY24,BTT,STEEL,1.0


In [10]:
# X-MC and X-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('X-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['X-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSP,FTT%,ATH%,Gained%,CumlRnk,Criteria,Strategy,Category,InFolio
11,ALKYLAMINE,4567.52,-11.54,-21.33,X-SC,9.65,167.95,167.95,8.67,63.0,SR,ATH,CHEMICALS,1.0
344,VAIBHAVGBL,521.00,-12.86,55.92,X-SC,69.50,117.42,288.83,19.46,46.0,XR,NTT,JEWELLERY,1.0
220,METROPOLIS,3102.00,-17.11,-10.62,X-SC,13.51,92.11,112.53,8.47,60.0,XR,NTT,HEALTHCARE,NaN
333,TTKPRESTIG,1149.24,-20.84,-9.79,X-SC,5.79,82.39,82.39,7.42,169.0,X40N,ATH,DURABLES,1.0
28,ASTRAL,2412.74,-20.11,-27.55,X-MC,40.93,81.94,81.94,7.11,38.0,AR,ATH,PLASTICS,NaN
147,HONAUT,58518.52,-15.01,-26.08,X-MC,58.30,62.24,62.24,10.48,73.0,X40N,ATH,ELECTRICAL,1.0
79,CERA,10420.45,-9.95,-17.90,X-SC,90.73,60.07,60.07,28.09,138.0,X40N,ATH,CERAMICS,1.0
91,DABUR,735.00,-13.93,-15.63,X-MC,17.95,56.98,41.85,3.72,107.0,XY24,BTT,FMCG,1.0
96,DEEPAKNTR,3102.60,-20.35,14.20,X-SC,41.31,56.63,56.63,9.60,80.0,AR,ATH,CHEMICALS,NaN
319,TATAELXSI,9000.00,-7.54,-18.84,X-MC,99.61,47.72,72.33,28.21,44.0,X40N,NTT,IT,1.0


In [11]:
# H-MC and H-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('H-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['H-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSP,FTT%,ATH%,Gained%,CumlRnk,Criteria,Strategy,Category,InFolio
312,TANLA,1963.11,-17.25,-39.81,H-SC,95.75,248.38,248.38,34.89,20.0,AR,ATH,IT,1.0
262,RELAXO,1176.00,-32.69,-39.88,H-SC,42.08,176.90,235.16,6.20,83.0,X40N,NTT,FOOTWEAR,1.0
348,VALIANTORG,838.00,-4.93,-300.87,H-SC,4.25,155.49,464.22,42.95,158.0,XR,NTT,CHEMICALS,1.0
48,BANDHANBNK,400.00,-5.04,-37.04,H-SC,71.81,146.34,339.10,24.61,30.0,XY24,NTT,BANKS,1.0
123,GLAND,3442.00,-14.94,-14.82,H-SC,29.34,138.30,195.77,3.70,69.0,XR,NTT,PHARMA,NaN
163,INDUSINDBK,1800.00,-28.37,-40.59,H-MC,52.90,133.54,152.14,20.99,24.0,XR,NTT,BANKS,1.0
270,REPCOHOME,880.00,-7.99,-56.78,H-SC,80.31,118.36,109.24,29.48,86.0,XY24,NTT,FINANCE,1.0
300,SONATSOFTW,852.37,-27.43,-4.50,H-SC,96.14,115.90,115.90,31.12,10.0,MGC,ATH,IT,NaN
159,INDIAMART,4911.36,-6.69,-53.52,H-SC,63.32,115.19,115.19,18.52,3.0,AR,ATH,MISC,1.0
357,VIPIND,718.00,-20.15,-791.15,H-SC,98.65,112.58,119.11,29.90,84.0,X40N,NTT,MISC,1.0


In [12]:
# x40
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSP,FTT%,ATH%,Gained%,CumlRnk,Criteria,Strategy,Category,InFolio
23,ASIANPAINT,4250.00,-13.01,-21.95,X-LC,5.41,86.18,53.22,7.06,113.0,X40,BTT,PAINTS,1.0
360,WHIRLPOOL,2270.00,-24.05,-35.67,M-SC,91.89,77.79,107.68,38.53,26.0,X40,NTT,DURABLES,1.0
54,BATAINDIA,2096.00,-10.35,-33.06,M-SC,15.44,72.81,79.38,2.71,76.0,X40,NTT,FOOTWEAR,1.0
42,BAJAJ-AUTO,12666.40,-13.34,39.59,X-LC,26.06,56.94,56.94,10.46,25.0,X40,ATH,AUTO,NaN
88,COLPAL,3808.06,-12.02,5.82,X-MC,45.17,46.86,46.86,10.31,18.0,X40,ATH,FMCG,1.0
243,PFIZER,6318.30,-11.92,-12.43,H-SC,54.44,43.52,43.52,12.48,110.0,X40,ATH,PHARMA,1.0
169,INFY,2275.00,-11.43,-11.34,X-LC,81.85,43.18,25.85,15.31,11.0,X40,BTT,IT,1.0
323,TCS,4998.00,-11.84,-15.19,X-LC,76.06,40.85,27.72,9.81,5.0,X40,BTT,IT,1.0
155,ICICIPRULI,791.05,-8.43,-15.03,H-MC,65.25,30.11,30.11,13.31,159.0,X40,ATH,INSURANCE,1.0
135,HAVELLS,2077.12,-5.48,-3.49,X-LC,34.36,29.96,29.96,13.59,78.0,X40,ATH,ELECTRICAL,1.0


In [13]:
# x40n
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40N'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSP,FTT%,ATH%,Gained%,CumlRnk,Criteria,Strategy,Category,InFolio
257,RAJESHEXPO,583.00,-16.48,1790.15,M-SC,32.82,199.59,387.85,28.25,196.0,X40N,NTT,JEWELLERY,1.0
262,RELAXO,1176.00,-32.69,-39.88,H-SC,42.08,176.90,235.16,6.20,83.0,X40N,NTT,FOOTWEAR,1.0
357,VIPIND,718.00,-20.15,-791.15,H-SC,98.65,112.58,119.11,29.90,84.0,X40N,NTT,MISC,1.0
1,5PAISA,810.00,-14.39,135.58,H-SC,35.14,110.25,91.06,21.99,179.0,X40N,BTT,FINANCE,1.0
162,INDIGOPNTS,2061.00,-24.59,87.50,H-SC,18.53,107.99,212.57,6.55,36.0,X40N,NTT,PAINTS,1.0
176,JCHAC,3542.00,-9.13,19769.57,M-SC,11.97,104.29,85.68,41.73,207.0,X40N,BTT,AC,1.0
333,TTKPRESTIG,1149.24,-20.84,-9.79,X-SC,5.79,82.39,82.39,7.42,169.0,X40N,ATH,DURABLES,1.0
113,FINCABLES,1657.02,-19.63,11.35,M-SC,43.82,79.44,79.44,17.61,133.0,X40N,ATH,CABLES,NaN
109,EQUITASBNK,113.10,-7.28,201.89,M-SC,79.15,76.77,76.77,16.52,199.0,X40N,ATH,BANKS,NaN
308,SUNTV,1092.00,-11.94,7.20,M-SC,1.16,75.89,44.57,11.16,144.0,X40N,BTT,ENTERTAINMENT,NaN


In [14]:
# xr, sr, ar
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XR','SR', 'AR'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSP,FTT%,ATH%,Gained%,CumlRnk,Criteria,Strategy,Category,InFolio
44,BAJAJHIND,442.93,-31.06,NaN,NaN,82.63,1972.67,1972.67,22.04,NaN,XR,ATH,SUGAR,NaN
78,CENTRALBK,190.06,-28.94,-42.02,L-MC,54.05,414.37,414.37,9.13,7.0,XR,ATH,BANKS,NaN
15,ALOKINDS,79.81,-16.71,NaN,L-SC,75.10,356.32,356.32,17.54,NaN,AR,ATH,TEXTILES,NaN
18,APEX,849.16,-10.20,NaN,NaN,20.85,292.35,292.35,10.00,NaN,AR,ATH,MISC,NaN
312,TANLA,1963.11,-17.25,-39.81,H-SC,95.75,248.38,248.38,34.89,20.0,AR,ATH,IT,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,MANAPPURAM,240.38,19.27,109.57,M-SC,42.86,3.35,3.35,68.62,232.0,XR,ATH,FINANCE,NaN
142,HINDPETRO,410.00,4.55,170.56,L-MC,53.67,3.20,12.89,35.25,216.0,XR,BTT,REFINERIES,NaN
216,MAXHEALTH,1208.75,14.21,56.19,X-LC,75.68,3.15,3.15,56.14,214.0,AR,NaN,HEALTHCARE,NaN
161,INDIGO,5558.50,19.29,48.97,M-LC,48.65,1.58,1.58,42.17,NaN,AR,ATH,AIRLINES,NaN
